In [6]:
# Módulos
from pyomo.environ import *
from pyomo.opt import SolverFactory

In [7]:
# Dados
Tarefas = [i for i in range(6)]
proc_tempo = {0: 0, 1: 64, 2: 53, 3: 63, 4: 99, 5: 189}
M = 10**6

In [8]:
# Modelo
model = ConcreteModel()

# Conjuntos
model.T = Set(initialize=Tarefas)
T = model.T

# Parâmetros
model.p = Param(T, initialize=proc_tempo, within=NonNegativeReals)
p = model.p

# Variáveis de decisão
model.x = Var(T * T, within=Binary)
model.C = Var(T, within=NonNegativeReals)
x, C = model.x, model.C

# Função objetivo
model.obj = Objective(sense=minimize,
                      expr = sum(C[i] for i in T))

# Restrições
# Um sucessor por tarefa
model.r_sucessor = ConstraintList()
for i in T:
    model.r_sucessor.add(expr = sum(x[i, j] for j in T if j != i) == 1)
# Um antecessor por tarefa
model.r_antecessor = ConstraintList()
for j in T:
    model.r_antecessor.add(expr = sum(x[i, j] for i in T if i != j) == 1)
# Tempo de término
model.r_termino = ConstraintList()
for i in T:
    for j in T:
        if j != i and j > 0:
            model.r_termino.add(expr = C[j] >= C[i] + p[j] * x[i, j] + M * (x[i, j] - 1))

# model.pprint()

In [9]:
# Resolução
solver = SolverFactory('glpk')
resultado = solver.solve(model, tee=False)

In [10]:
# Apresentação de resultados
if (resultado.solver.status == SolverStatus.ok) and (resultado.solver.termination_condition != TerminationCondition.infeasible):
    for i in T:
        for j in T:
            if i != j and value(x[i,j]) == 1:
                print('A tarefa {} precede a {}. A tarefa {} é completada no minuto {:.0f}.'.format(i, j, i, value(C[i])))
    print()
    print('Tempo de fluxo total {:.0f} minutos'.format(value(model.obj)))
else:
    print(resultado.solver.termination_condition)

A tarefa 0 precede a 2. A tarefa 0 é completada no minuto 0.
A tarefa 1 precede a 4. A tarefa 1 é completada no minuto 180.
A tarefa 2 precede a 3. A tarefa 2 é completada no minuto 53.
A tarefa 3 precede a 1. A tarefa 3 é completada no minuto 116.
A tarefa 4 precede a 5. A tarefa 4 é completada no minuto 279.
A tarefa 5 precede a 0. A tarefa 5 é completada no minuto 468.

Tempo de fluxo total 1096 minutos
